In [ ]:
from __future__ import annotations

import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

In [ ]:
import numpy as np
import requests

In [ ]:
API_KEY = os.getenv("EURI_API_KEY")

In [ ]:
def generate_embeddings(text: str) -> np.ndarray:
	url = "https://api.euron.one/api/v1/euri/embeddings"
	headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
	payload = {"input": text, "model": "text-embedding-3-small"}

	response = requests.post(url, headers=headers, json=payload, timeout=30)
	data = response.json()

	# convert to numpy array for vector operations

	return np.array(data["data"][0]["embedding"])

In [ ]:
text = "The weather is sunny today."

embedding = generate_embeddings(text)

In [ ]:
embedding

In [ ]:
print(f"Embedding shape: {embedding.shape}")

In [ ]:
# calculate vector norm
norm = np.linalg.norm(embedding)
print(f"vector norm: {norm}")

In [ ]:
def generate_completion():
	url = "https://api.euron.one/api/v1/euri/chat/completions"
	headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
	payload = {
		"messages": [
			{"role": "user", "content": "Write a poem about artificial intelligence"}
		],
		"model": "gpt-4.1-nano",
		"max_tokens": 500,
		"temperature": 0.7,
	}

	response = requests.post(url, headers=headers, json=payload, timeout=30)
	data = response.json()
	print(data)

In [ ]:
generate_completion()

In [ ]:
from pathlib import Path

with Path("../data/founder_story.txt").open(encoding="utf-8") as f:
	raw_text = f.read()

In [ ]:
raw_text

### Here chunk means it has approx. 100 words in each chunk


In [ ]:
def chunk_text(text, max_words=100):
	words = text.split()
	return [" ".join(words[i : i + max_words]) for i in range(0, len(words), max_words)]

In [ ]:
chunks = chunk_text(raw_text)
print(f"Total number of chunks: {len(chunks)}")
print(chunks[0])

In [ ]:
embedding = generate_embeddings(chunks[0])
print(embedding)

In [ ]:
import faiss

In [ ]:
dimension = embedding.shape[0]
index = faiss.IndexFlatL2(dimension)

In [ ]:
chunk_mapping = []
for chunk in chunks:
	embedding = generate_embeddings(chunk)
	index.add(np.array([embedding]).astype("float32"))
	chunk_mapping.append(chunk)

In [ ]:
faiss.write_index(index, "../data/index.faiss")

In [ ]:
def retrieve_top_k(query, k=1):
	query_embedding = generate_embeddings(query)
	_distance, indices = index.search(np.array([query_embedding]).astype("float32"), k)
	return [chunk_mapping[i] for i in indices[0]]

In [ ]:
def build_prompt(context_chunks, query):
	context = "\n\n".join(context_chunks)
	return f"""Use the following context to answer the question.
	Context:
	{context}

	Question:
	{query}

	Answer:"""

In [ ]:
def generate_response(prompt, model="gpt-4.1-nano"):
	url = "https://api.euron.one/api/v1/euri/chat/completions"
	headers = {"Content-Type": "application/json", "Authorization": f"Bearer {API_KEY}"}
	payload = {
		"messages": [{"role": "user", "content": prompt}],
		"model": model,
		"max_tokens": 500,
		"temperature": 0.3,
	}
	response = requests.post(url, headers=headers, json=payload, timeout=30)
	return response.json()["choices"][0]["message"]["content"]

In [ ]:
query = "what was the birth place of Sudhanshu?"
top_chunks = retrieve_top_k(query=query, k=3)

In [ ]:
final_prompt = build_prompt(context_chunks=top_chunks, query=query)

In [ ]:
result = generate_response(prompt=final_prompt)
result